In [ ]:
# default_exp core

# kicad-helpers

> Scripts, templates, and examples for managing KiCad projects.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sylvain")

'Hello Sylvain!'

In [ ]:
# Examples can output plots, images, etc, and they'll all appear in your docs, e.g.:

from IPython.display import display,SVG

display(SVG('<svg height="100"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
# You can also include tests:

assert say_hello("Jeremy")=="Hello Jeremy!"